In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
#     ('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
#     ('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
#     ('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
#     ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
#     ('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
#     ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
#     ('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
#     ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
    #('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
    
    ('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4465


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(2, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0035
Function value obtained: 0.4747
Current minimum: 0.4747
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0017
Function value obtained: 0.4748
Current minimum: 0.4747
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0015
Function value obtained: 0.4750
Current minimum: 0.4747
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0015
Function value obtained: 0.4753
Current minimum: 0.4747
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.5031
Function value obtained: 0.4746
Current minimum: 0.4746
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.4850
Function value obtained: 0.4751
Current minimum: 0.4746
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.5098
Function value obtained: 0.4756
Current minimum: 0.4746
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.4841
Function value obtained: 0.4746
Current minimum: 0.4746
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.4790
Function value obtained: 0.4746
Current minimum: 0.4746
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.6933
Function value obtained: 0.4754
Current minimum: 0.4746
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.7526
Function value obtained: 0.4748
Current minimum: 0.4746
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.7045
Function value obtained: 0.4752
Current minimum: 0.4746
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.7497
Function value obtained: 0.4747
Current minimum: 0.4746
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.7565
Function value obtained: 0.4751
Current minimum: 0.4746
Iteration No: 86 started. Sea

Current minimum: 0.4405
Iteration No: 21 started. Searching for the next optimal point.
Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.2095
Function value obtained: 0.4407
Current minimum: 0.4405
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.2652
Function value obtained: 0.4407
Current minimum: 0.4405
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.2735
Function value obtained: 0.4411
Current minimum: 0.4405
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.2499
Function value obtained: 0.4408
Current minimum: 0.4405
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.27

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.5688
Function value obtained: 0.4408
Current minimum: 0.4405
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.5787
Function value obtained: 0.4411
Current minimum: 0.4405
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.5919
Function value obtained: 0.4405
Current minimum: 0.4405
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.5916
Function value obtained: 0.4405
Current minimum: 0.4405
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 0.6034
Function value obtained: 0.4405
Current minimum: 0.4405
Iteration No: 66 started. Sea

...

Iteration No: 87 ended. Search finished for the next optimal point.
Time taken: 0.8156
Function value obtained: 0.4384
Current minimum: 0.4384
Iteration No: 88 started. Searching for the next optimal point.
Iteration No: 88 ended. Search finished for the next optimal point.
Time taken: 0.8069
Function value obtained: 0.4384
Current minimum: 0.4384
Iteration No: 89 started. Searching for the next optimal point.


/home/takesako/anaconda3/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 89 ended. Search finished for the next optimal point.
Time taken: 0.8461
Function value obtained: 0.4384
Current minimum: 0.4384
Iteration No: 90 started. Searching for the next optimal point.
Iteration No: 90 ended. Search finished for the next optimal point.
Time taken: 0.8870
Function value obtained: 0.4386
Current minimum: 0.4384
Iteration No: 91 started. Searching for the next optimal point.
Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 0.9322
Function value obtained: 0.4389
Current minimum: 0.4384
Iteration No: 92 started. Searching for the next optimal point.
Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 0.9899
Function value obtained: 0.4384
Current minimum: 0.4384
Iteration No: 93 started. Searching for the next optimal point.
Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 0.9255
Function value obtained: 0.4393
Current minimum: 0.4384
Iteration No: 94 started. Sea

/home/takesako/anaconda3/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 95 ended. Search finished for the next optimal point.
Time taken: 0.9951
Function value obtained: 0.4384
Current minimum: 0.4384
Iteration No: 96 started. Searching for the next optimal point.
Iteration No: 96 ended. Search finished for the next optimal point.
Time taken: 0.9235
Function value obtained: 0.4395
Current minimum: 0.4384
Iteration No: 97 started. Searching for the next optimal point.
Iteration No: 97 ended. Search finished for the next optimal point.
Time taken: 0.9801
Function value obtained: 0.4404
Current minimum: 0.4384
Iteration No: 98 started. Searching for the next optimal point.
Iteration No: 98 ended. Search finished for the next optimal point.
Time taken: 1.0471
Function value obtained: 0.4384
Current minimum: 0.4384
Iteration No: 99 started. Searching for the next optimal point.
Iteration No: 99 ended. Search finished for the next optimal point.
Time taken: 1.0029
Function value obtained: 0.4384
Current minimum: 0.4384
Iteration No: 100 started. Se

In [10]:
best_weights

{'cohesion': array([0.79711893, 0.20288107]),
 'syntax': array([0.1631727, 0.8368273]),
 'vocabulary': array([0.09090909, 0.90909091]),
 'phraseology': array([0.41482563, 0.58517437]),
 'grammar': array([0.33851197, 0.66148803]),
 'conventions': array([0.09090909, 0.90909091])}

In [11]:
scores

{'cohesion': 0.47458443978280457,
 'syntax': 0.4404722328677637,
 'vocabulary': 0.4078945872047041,
 'phraseology': 0.4498688924837863,
 'grammar': 0.4655476638002069,
 'conventions': 0.43839830310946704}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4461


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.949500,0.915472,0.916242,0.828887,0.893219
syntax,0.949500,1.000000,0.940836,0.966137,0.920249,0.931378
vocabulary,0.915472,0.940836,1.000000,0.963411,0.877349,0.894701
phraseology,0.916242,0.966137,0.963411,1.000000,0.942677,0.890074
grammar,0.828887,0.920249,0.877349,0.942677,1.000000,0.856534
conventions,0.893219,0.931378,0.894701,0.890074,0.856534,1.000000
